In [ ]:
import pandas as pd
import json

exp_name = 'mols'

runs_pd = pd.read_csv(f"{exp_name}.csv")
run_dict = {}
for (sum, name) in zip(runs_pd["summary"], runs_pd["name"]):
    sum_dict = eval(sum)
    if exp_name == 'mols':
        if sum_dict['step'] != 49999:
            continue
        if sum_dict['random_action_prob'] != 0.05:
            continue
        if sum_dict['use_exp_weight_decay'] != False:
            continue
    if exp_name == 'bit':
        if sum_dict['step'] != 49999:
            continue
        if sum_dict['grad_clip_norm'] != 20:
            continue
        if sum_dict['use_exp_weight_decay'] != False:
            continue
    if exp_name == 'set':
        if sum_dict['step'] != 9999:
            continue
        if sum_dict['training_mode'] != 'online':
            continue
        if sum_dict['use_alpha_scheduler'] != True:
            continue
        if sum_dict['use_grad_clip'] != False:
            continue
        # if sum_dict['fl'] != True:
        #     continue
    run_dict[name] = sum_dict

json.dump(run_dict, open(f"{exp_name}.json", "w"), indent=2)
print(len(run_dict))

225


In [ ]:
cnt = 0
res = []
for key, value in run_dict.items():
    if value['fl'] == False and value['objective'] == 'tb':
        cnt += 1
        res.append((cnt, value['alpha_init'], value['alpha']))
res.sort(key=lambda x: x[1])
res

[(4, 0.1, 0.4926708133561055),
 (5, 0.1, 0.4926708133561055),
 (6, 0.1, 0.4926708133561055),
 (12, 0.1, 0.4926708133561055),
 (15, 0.1, 0.4926708133561055),
 (7, 0.2, 0.49450311001707914),
 (8, 0.2, 0.49450311001707914),
 (11, 0.2, 0.49450311001707914),
 (13, 0.2, 0.49450311001707914),
 (16, 0.2, 0.49450311001707914),
 (1, 0.3, 0.4963354066780528),
 (3, 0.3, 0.4963354066780528),
 (9, 0.3, 0.4963354066780528),
 (17, 0.3, 0.4963354066780528),
 (18, 0.3, 0.4963354066780528),
 (2, 0.4, 0.4981677033390264),
 (14, 0.4, 0.4981677033390264),
 (19, 0.4, 0.4981677033390264),
 (20, 0.4, 0.4981677033390264),
 (35, 0.4, 0.4981677033390264),
 (23, 0.5, 0.5),
 (28, 0.5, 0.5),
 (30, 0.5, 0.5),
 (34, 0.5, 0.5),
 (40, 0.5, 0.5),
 (21, 0.6, 0.5018322966609736),
 (22, 0.6, 0.5018322966609736),
 (33, 0.6, 0.5018322966609736),
 (38, 0.6, 0.5018322966609736),
 (41, 0.6, 0.5018322966609736),
 (10, 0.7, 0.5036645933219472),
 (25, 0.7, 0.5036645933219472),
 (31, 0.7, 0.5036645933219472),
 (32, 0.7, 0.5036645933

In [15]:
exp_name = 'set'
runs_pd = pd.read_csv(f"{exp_name}.csv")
sum = eval(runs_pd["summary"][100])
sum['step']

9999

In [26]:
# Set table vs mode
import pandas as pd
import ast

# 读取 set.csv
df = pd.read_csv("set.csv")

# sum_dict['step'] != 49999:  # 只保留 step 为 49999 的行
df = df[df["summary"].apply(lambda x: ast.literal_eval(x).get("step", None) == 9999)]

# 解析 summary 字典
df["summary_dict"] = df["summary"].apply(lambda x: ast.literal_eval(x))

# 从 summary_dict 中提取需要的字段
df["alpha"] = df["summary_dict"].apply(lambda d: float(d.get("alpha", float("nan"))))
df["modes"] = df["summary_dict"].apply(lambda d: float(d.get("modes", float("nan"))))
df["mean_R"] = df["summary_dict"].apply(lambda d: float(d.get("mean_R", float("nan"))))

# 定义行和列标签
rows = ["Baseline", "Ours"]
cols = ["Modes", "Mean R"]

# 初始化表格内容
table = {row: {col: "" for col in cols} for row in rows}

# Baseline: alpha == 0.5
baseline_row = df.iloc[(df["alpha"] - 0.5).abs().argsort()[:1]]  # 最接近0.5的
if not baseline_row.empty:
    baseline_row = baseline_row.iloc[0]
    table["Baseline"]["Modes"] = f"{baseline_row['modes']:.2f}"
    table["Baseline"]["Mean R"] = f"{baseline_row['mean_R']:.2f}"
else:
    table["Baseline"]["Modes"] = "N/A"
    table["Baseline"]["Mean R"] = "N/A"

# Ours: max modes
ours_row = df.loc[df["modes"].idxmax()]
table["Ours"]["Modes"] = f"{ours_row['modes']:.2f}"
table["Ours"]["Mean R"] = f"{ours_row['mean_R']:.2f}"

# 输出 LaTeX 表格
print("\\begin{tabular}{lcc}")
print("\\toprule")
print("Method & Modes & Mean R \\\\")
print("\\midrule")
for row in rows:
    line = row
    for col in cols:
        val = table[row][col]
        if row == "Ours":
            val = f"\\\\textbf{{{val}}}"  # 加粗 Ours
        line += f" & {val}"
    print(line + " \\\\")
print("\\bottomrule")
print("\\end{tabular}")


\begin{tabular}{lcc}
\toprule
Method & Modes & Mean R \\
\midrule
Baseline & 0.00 & 1233.73 \\
Ours & \\textbf{4365.00} & \\textbf{360754.53} \\
\bottomrule
\end{tabular}


In [50]:
import pandas as pd
import ast
import numpy as np

def csv_to_latex_table(csv_path: str, k:int=6, methods_order=None, T=True):
    # 读取csv
    df = pd.read_csv(csv_path)

    # sum_dict['step'] != 9999:  # 只保留 step 为 9999 的行
    df = df[df["summary"].apply(lambda x: ast.literal_eval(x).get("step", None) == 49999)]
    # 过滤 M_size = k
    df = df[df["summary"].apply(lambda x: ast.literal_eval(x).get("k", None) == k)]

    # 解析 summary 列
    def parse_summary(s):
        try:
            return ast.literal_eval(s)
        except Exception:
            return {}
    
    df["parsed"] = df["summary"].apply(parse_summary)
    
    # 提取关键字段
    df["alpha"] = df["parsed"].apply(lambda x: x.get("alpha_init", None))
    df["modes"] = df["parsed"].apply(lambda x: x.get("modes", None))
    df["objective"] = df["parsed"].apply(lambda x: x.get("objective", None))
    
    # 按 alpha, method 分组，计算均值和标准差
    grouped = df.groupby(["alpha", "objective"])["modes"].agg(["mean", "std"]).reset_index()
    
    # 生成透视表
    table = grouped.pivot(index="alpha", columns="objective", values=["mean", "std"])
    
    # 格式化 "均值 ± 标准差"
    def format_val(m, s):
        if pd.isna(m): 
            return "-"
        return f"{m:.2f} ± {s:.2f}"
    
    methods = methods_order if methods_order else sorted(df["objective"].unique())
    alphas = sorted(df["alpha"].unique())
    

    if T == True:
        # 生成latex表格
        latex = []
        latex.append("\\begin{table}[h]")
        latex.append("\\centering")
        latex.append(f"\\caption{{Results for $k={k}$}}")
        latex.append("\\begin{tabular}{c" + "c"*len(alphas) + "}")
        latex.append("\\toprule")
        
        # 表头
        header = "Method & " + " & ".join([f"{a:.1f}" for a in alphas]) + " \\\\"
        latex.append(header)
        latex.append("\\midrule")
        
        # 表格内容
        for m in methods:
            row = [m]
            for a in alphas:
                mean = table.loc[a, ("mean", m)] if (a, m) in grouped.set_index(["alpha","objective"]).index else np.nan
                std = table.loc[a, ("std", m)] if (a, m) in grouped.set_index(["alpha","objective"]).index else np.nan
                row.append(format_val(mean, std))
            latex.append(" & ".join(row) + " \\\\")
        
        latex.append("\\bottomrule")
        latex.append("\\end{tabular}")
        latex.append("\\end{table}")
        
        return "\n".join(latex)
    else:
        # 生成latex表格
        latex = []
        latex.append("\\begin{table}[h]")
        latex.append("\\centering")
        latex.append("\\begin{tabular}{c" + "c"*len(methods) + "}")
        latex.append("\\toprule")
        
        # 表头
        header = "Alpha & " + " & ".join(methods) + " \\\\"
        latex.append(header)
        latex.append("\\midrule")

        
        # 表格内容
        for a in alphas:
            row = [f"{a:.1f}"]
            for m in methods:
                mean = table.loc[a, ("mean", m)] if (a, m) in grouped.set_index(["alpha","objective"]).index else np.nan
                std = table.loc[a, ("std", m)] if (a, m) in grouped.set_index(["alpha","objective"]).index else np.nan
                row.append(format_val(mean, std))
            latex.append(" & ".join(row) + " \\\\")
        
        latex.append("\\bottomrule")
        latex.append("\\end{tabular}")
        latex.append("\\end{table}")
        
        return "\n".join(latex)


# 使用示例
print(csv_to_latex_table("bit.csv", methods_order=['db', 'tb', 'subtb'], k=10))


\begin{table}[h]
\centering
\caption{Results for $k=10$}
\begin{tabular}{cccccccccc}
\toprule
Method & 0.1 & 0.2 & 0.3 & 0.4 & 0.5 & 0.6 & 0.7 & 0.8 & 0.9 \\
\midrule
db & 6.60 ± 2.30 & 7.40 ± 1.95 & 6.40 ± 2.30 & 7.60 ± 2.07 & 6.80 ± 0.84 & 6.00 ± 3.67 & 6.60 ± 2.88 & 6.60 ± 2.19 & 7.40 ± 1.52 \\
tb & 25.80 ± 3.96 & 22.40 ± 1.67 & 25.80 ± 3.70 & 23.80 ± 2.39 & 25.40 ± 4.83 & 26.00 ± 3.16 & 28.20 ± 3.49 & 24.60 ± 3.36 & 20.80 ± 4.82 \\
subtb & 19.40 ± 4.88 & 19.40 ± 2.70 & 19.00 ± 2.55 & 17.60 ± 3.78 & 19.60 ± 4.93 & 18.80 ± 3.96 & 19.40 ± 3.36 & 19.40 ± 2.19 & 18.60 ± 3.91 \\
\bottomrule
\end{tabular}
\end{table}
